In [1]:
from pyspark.sql import SparkSession
from pyspark import  SparkContext
import os
import pandas as pd

/tmp/ipykernel_22294/2081191832.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
spark = SparkSession.builder.master("yarn").getOrCreate()

/spark/bin/load-spark-env.sh: line 68: ps: command not found


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/21 03:23:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/21 03:23:36 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/03/21 03:23:37 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [3]:
import pandas as pd

# Sample data for Wineries table
wineries_data = {
    'id': [103, 737, 848, 222, 116, 124, 648, 894, 677],
    'country': ['Australia', 'Australia', 'Australia', 'Hungary', 'USA', 'USA', 'India', 'USA', 'USA'],
    'points': [84, 85, 100, 60, 47, 45, 69, 39, 9],
    'winery': ['WhisperingPines', 'GrapesGalore', 'HarmonyHill', 'MoonlitCellars', 'RoyalVines', 'Eagle\'sNest', 'SunsetVines', 'RoyalVines', 'PacificCrest']
}

# Create DataFrame for Wineries
wineries_df = pd.DataFrame(wineries_data)

# Display the DataFrame
print("Wineries table:")
print(wineries_df)



df_person = spark.createDataFrame(wineries_df)
df_person.createOrReplaceTempView("Wineries")



Wineries table:
    id    country  points           winery
0  103  Australia      84  WhisperingPines
1  737  Australia      85     GrapesGalore
2  848  Australia     100      HarmonyHill
3  222    Hungary      60   MoonlitCellars
4  116        USA      47       RoyalVines
5  124        USA      45      Eagle'sNest
6  648      India      69      SunsetVines
7  894        USA      39       RoyalVines
8  677        USA       9     PacificCrest


In [10]:
query = """
    with rk_points as (
        select
            *
        from (
            select
                *,
                row_number() over(partition by country order by points desc, winery asc) as rk
            from (
                select
                    country,
                    winery,
                    sum(points) as points
                from Wineries w 
                group by country, winery
            )
        )
        where rk <= 3
    ),
    
    r as (
        select
            r.country,
            r.rk,
            p.points,
            p.winery,
            case 
                when r.rk = 2 and p.points is null then 'No second winery'
                when r.rk = 3 and p.points is null then 'No third winery'
                else concat(p.winery, " (", p.points, ")") 
            end as name
        from (
            select
                distinct
                country,
                explode(sequence(1, 3)) as rk
            from Wineries w 
            order by country, rk
        ) r left join rk_points p on r.country = p.country and r.rk = p.rk
        order by country, rk
    )
    
    select * from (
        select
            country,
            rk,
            name
        from r
    )
    pivot (
        max(name) for rk in (
            1 top_winery,
            2 second_winery,
            3 third_winery
        )
    )
"""

In [11]:
spark.sql(query).show()

+---------+-------------------+-----------------+--------------------+
|  country|         top_winery|    second_winery|        third_winery|
+---------+-------------------+-----------------+--------------------+
|Australia|  HarmonyHill (100)|GrapesGalore (85)|WhisperingPines (84)|
|  Hungary|MoonlitCellars (60)| No second winery|     No third winery|
|    India|   SunsetVines (69)| No second winery|     No third winery|
|      USA|    RoyalVines (86)| Eagle'sNest (45)|    PacificCrest (9)|
+---------+-------------------+-----------------+--------------------+

